In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

In [ ]:
driver = webdriver.Chrome()
driver.get("https://webgate.ec.europa.eu/fleet-europa/search_en")

wait = WebDriverWait(driver, 1)

# Hacer clic en el botón "EU"
eu_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//label[@for='countryType1']")))
eu_option.click()

# Hacer click en "All Vessels"
all_vessels_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//label[@for='period1']")))
all_vessels_option.click()

# Hacer clic en el botón "Search"
btn_search = wait.until(EC.element_to_be_clickable((By.XPATH, "//button/span[text()='Search']")))
btn_search.click()


#-----------------------------------------------------------------------------------------------------#

# Esperar a que el selector de la página de resultados se muestre
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "select2-selection--single")))

page_size_selector = driver.find_element(By.CLASS_NAME, "select2-selection--single")
page_size_selector.click()

# Esperar a que la lista con opciones sea visible
wait.until(EC.presence_of_element_located((By.XPATH, "//ul[@class='select2-results__options']")))

# Intentar hacer clic en la opción que contiene el texto "100"
page_size_100 = wait.until(EC.element_to_be_clickable((By.XPATH, "//li[contains(text(),'100')]")))

# Hacer clic en la opción "100"
page_size_100.click()
#-----------------------------------------------------------------------------------------------------#
# Esperar a que la tabla cargue
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "table-header-container")))

# Nº de resultados por página 100
page_size_selector = driver.find_element(By.CLASS_NAME, "select2-selection--single")
page_size_selector.click()

# Extraer los nombres de las columnas 
column_headers = driver.find_elements(By.XPATH, "//div[@class='table-header-container']/span")
column_names = [header.text.strip() for header in column_headers] 

# Contenido de las filas
rows = driver.find_elements(By.XPATH, "//table/tbody/tr")

data = []

while True:
    # Extraer filas de la tabla
    rows = driver.find_elements(By.XPATH, "//table/tbody/tr")
    
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        row_data = [cell.text.strip() for cell in cells]  # Solo tomar los primeros 3 campos
        data.append(row_data)

    # Intentar encontrar el botón "Next"
    try:
        # Buscar el botón "Next" usando el atributo aria-label
        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@aria-label='Go to next page']")))

        # Desplazar hasta el botón "Next"
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        
        # Verificar si el botón "Next" está habilitado
        if "disabled" not in next_button.get_attribute("class"):  
            # Hacer clic en el botón "Next"
            next_button.click()
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "table")))  # Esperar a que la nueva página cargue
        else:
            break  # Si el botón está deshabilitado, salir del bucle
    except:
        break  # Si no hay botón "Next" o hubo un error, salir del bucle
   

# Crear DataFrame con todos los datos recopilados
df = pd.DataFrame(data, columns=column_names)

# Mostrar el DataFrame
print(df)

# Cerrar el driver
driver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=134.0.6998.177)
Stacktrace:
	GetHandleVerifier [0x00007FF6F59F4C25+3179557]
	(No symbol) [0x00007FF6F56588A0]
	(No symbol) [0x00007FF6F54E91CA]
	(No symbol) [0x00007FF6F54C1753]
	(No symbol) [0x00007FF6F556F83E]
	(No symbol) [0x00007FF6F558FBE2]
	(No symbol) [0x00007FF6F5567A03]
	(No symbol) [0x00007FF6F55306D0]
	(No symbol) [0x00007FF6F5531983]
	GetHandleVerifier [0x00007FF6F5A567CD+3579853]
	GetHandleVerifier [0x00007FF6F5A6D1D2+3672530]
	GetHandleVerifier [0x00007FF6F5A62153+3627347]
	GetHandleVerifier [0x00007FF6F57C092A+868650]
	(No symbol) [0x00007FF6F5662FFF]
	(No symbol) [0x00007FF6F565F4A4]
	(No symbol) [0x00007FF6F565F646]
	(No symbol) [0x00007FF6F564EAA9]
	BaseThreadInitThunk [0x00007FF8A1B2259D+29]
	RtlUserThreadStart [0x00007FF8A326AF38+40]


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Flag         25 non-null     object
 1   Vessel Type  25 non-null     object
 2   CFR          25 non-null     object
dtypes: object(3)
memory usage: 732.0+ bytes
